<a href="https://colab.research.google.com/github/novoblake/AlgortihmsAndDataStructures/blob/master/GCN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCN

This notebook accompanies my blog on Graph Neural Networks and takes inspiration from PyTorch Geometric tutorial.

In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl size=3536353 sha256=8138698cb08422155bfb841c2a4e4c7e4aa59038b19cc9964971c74ef3ee1a23
  Stored in directory: /root/.cache/pip/wheels/ef/67/58/6566a3b61c6ec0f2ca0c2c324cd035ef2955601f0fb3197d5f
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.17-cp310-cp310-linux_x86_64.whl size=2693695 sha256=ab5b058358c5fff3aec76a60c769ccc7ac07a53db8bd39f2564f80f94956027d
  Stored in directory: /root/.cache/pip/wheels/67/25/e7/037b58fa47ba781444fd101a2f06c63a9d4e967ca6b910c53a
Successfully built t

In [ ]:
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import ChebConv, GCNConv

In [ ]:
dataset = 'Cora'
path = "."
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True,
                             normalize=True)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.3143, Val: 0.1400, Test: 0.1580
Epoch: 002, Train: 0.5571, Val: 0.3800, Test: 0.3800
Epoch: 003, Train: 0.7571, Val: 0.5400, Test: 0.5580
Epoch: 004, Train: 0.7929, Val: 0.6280, Test: 0.6270
Epoch: 005, Train: 0.8143, Val: 0.6280, Test: 0.6270
Epoch: 006, Train: 0.8714, Val: 0.6540, Test: 0.6600
Epoch: 007, Train: 0.9071, Val: 0.6720, Test: 0.6960
Epoch: 008, Train: 0.9143, Val: 0.6920, Test: 0.7080
Epoch: 009, Train: 0.9143, Val: 0.7060, Test: 0.7270
Epoch: 010, Train: 0.9214, Val: 0.7100, Test: 0.7280
Epoch: 011, Train: 0.9357, Val: 0.7280, Test: 0.7280
Epoch: 012, Train: 0.9357, Val: 0.7280, Test: 0.7280
Epoch: 013, Train: 0.9214, Val: 0.7280, Test: 0.7280
Epoch: 014, Train: 0.9214, Val: 0.7280, Test: 0.7280
Epoch: 015, Train: 0.9214, Val: 0.7280, Test: 0.7280
Epoch: 016, Train: 0.9286, Val: 0.7300, Test: 0.7240
Epoch: 017, Train: 0.9286, Val: 0.7300, Test: 0.7240
Epoch: 018, Train: 0.9214, Val: 0.7320, Test: 0.7310
Epoch: 019, Train: 0.9214, Val: 0.7360, Test: 